In [3]:
%matplotlib tk
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns
import math 
import time
from IPython.display import display, clear_output

Create marker stream

In [ ]:
from pylsl import StreamInfo, StreamOutlet, IRREGULAR_RATE

info = StreamInfo(
    'TimeSeriesMarkerStream',
    'Markers',
    1,
    IRREGULAR_RATE,
    'int32',
    'eegmarker1'
)

marker_index = 1000

outlet = StreamOutlet(info)

df_marker

def push(id = None):
    outlet.push_sample([marker_index])
    return marker_index
    

print("Marker Streaming ready...")


In [4]:
window_size = 100
display_delay = 4

inp_normal_path = 'NeurIPS-ts/data/normal.csv'
normal_no = 0
inp_anomal_path = 'NeurIPS-ts/data/01234.csv'
sample_no = 3
out_path = 'TS-data/0.csv'

In [5]:
df_normal = pd.read_csv(inp_normal_path)
df = pd.read_csv(inp_anomal_path)
df

,value,anomaly
0,-0.131232,0
1,0.398736,0
2,0.809108,0
3,1.007888,0
4,1.340091,0
...,...,...
995,-5.981934,0
996,-5.667016,0
997,-5.457295,0
998,-5.206649,0


In [6]:
#run normal on sequence
stride = int(window_size/2)
starts = []
for i in range(normal_no):
    starts.append(stride*i)
samples_normal = df_normal.loc[starts]
samples_normal['start_point'] = starts
samples_normal.drop(columns=['value'], inplace=True)
samples_normal = samples_normal.reset_index()

In [7]:
#run with random start points
#start_points = np.random.randint(df.shape[0]-window_size, size=sample_no)
#samples = df.loc[start_points]
#samples['start_point'] = start_points
#samples.drop(columns=['value'], inplace=True)
#samples = samples.reset_index()

In [8]:
#run based on sequence with stride = window/2
stride = int(window_size/2)
start_points = []
for i in range(sample_no):
    start_points.append(stride*i)
samples = df.loc[start_points]
samples['start_point'] = start_points
samples.drop(columns=['value'], inplace=True)
samples = samples.reset_index()

In [11]:
dataset_ts = pd.DataFrame(columns=['ID','Start','End','Label','Percentage'])
ID = 1

fig = plt.figure()
ax = fig.add_subplot(1, 1, 1) 

for i in range(normal_no):
    ax.cla()
    ax.axes.get_yaxis().set_visible(False)
    ax.axes.get_xaxis().set_visible(False)
    clear_output(wait = True)
    plt.show()
    plt.pause(display_delay)
    
    ax.plot(df_normal[samples_normal['start_point'][i]:samples_normal['start_point'][i]+window_size].reset_index()['value'])
    clear_output(wait = True)
    plt.show()
    plt.pause(display_delay)

for i in range(sample_no):
    ax.cla()
    ax.axes.get_yaxis().set_visible(False)
    ax.axes.get_xaxis().set_visible(False)
    clear_output(wait = True)
    plt.show()
    plt.pause(display_delay)
    ls = []
    ls.append(ID)
    ls.append(samples['start_point'][i])
    ls.append(samples['start_point'][i]+window_size)
    slide_win = df[samples['start_point'][i]:samples['start_point'][i]+window_size]
    if (slide_win['anomaly'] == 1).sum()>0 :
        ls.append(1)
        ls.append((slide_win['anomaly'] == 1).sum() / window_size)
    else:
        ls.append(0)
        ls.append(0)
    dataset_ts.loc[len(dataset_ts)] = ls  
        
    ax.plot(slide_win.reset_index()['value'])
    clear_output(wait = True)
    plt.show()
    plt.pause(display_delay)
    ID = ID + 1

plt.close()
print(dataset_ts)
dataset_ts.to_csv(out_path)

    ID  Start    End  Label  Percentage
0  1.0    0.0  100.0    1.0        0.35
1  2.0   50.0  150.0    1.0        0.35
2  3.0  100.0  200.0    1.0        0.17
